In [12]:
import numpy as np
import pandas as pd

In [30]:
from math import sqrt

class Point:
    
    def __init__(self, coordinates, label):
        self.coordinates = coordinates
        self.label = label
    
    def get_coordinates(self):
        return self.coordinates
    
    def get_label(self):
        return self.label
    
    def distance(self, point):
        return sqrt(sum([(x - y) ** 2 for x, y in zip(self.coordinates, point.coordinates)]))
    
    def __str__(self):
        return str(self.coordinates) + " " + str(self.label)

In [31]:
class Model:
    
    def __init__(self, k=7):
        self.training_set = None
        self.k = k
    
    def train(self, training_set):
        self.training_set = training_set
    
    def classify(self, test_set):
        
        res = []
        
        for point in test_set:
            neighbours = sorted(self.training_set, key = lambda x: x.distance(point))[0:self.k]
            labels = [x.label for x in neighbours]
            prediction = max(labels, key = labels.count)
            res += [(point, prediction)] 
        return res
    
    def confusion(self, test_set):
        
        predictions = self.classify(test_set)
        rows = list(set([x.label for x in self.training_set]))
        columns = list(set([x[1] for x in predictions]))
        res = ""
        

In [ ]:
class ConfusionMatrix:
    
    def __init__(self, predictions):
        self.predicitions = predicitions
        
    def __str__(self):
        

In [32]:
training_set = [Point(x[0:-1], x[-1]) for x in pd.read_csv("data/training_set.csv").values.tolist()]
test_set     = [Point(x[0:-1], x[-1]) for x in pd.read_csv("data/test_set.csv").values.tolist()]
model = Model()
model.train(training_set)
predictions = model.classify(test_set)

for prediction in predictions:
    print(prediction[0], prediction[1])

[4.4, 3.2, 1.3, 0.2] 1.0 1.0
[4.6, 3.2, 1.4, 0.2] 1.0 1.0
[6.4, 2.9, 4.3, 1.3] 2.0 2.0
[4.9, 3.1, 1.5, 0.1] 1.0 1.0
[4.6, 3.6, 1.0, 0.2] 1.0 1.0
[6.3, 2.7, 4.9, 1.8] 3.0 3.0
[6.3, 2.5, 4.9, 1.5] 2.0 3.0
[4.8, 3.4, 1.9, 0.2] 1.0 1.0
[6.1, 2.9, 4.7, 1.4] 2.0 2.0
[5.8, 2.6, 4.0, 1.2] 2.0 2.0
[5.1, 3.3, 1.7, 0.5] 1.0 1.0
[7.3, 2.9, 6.3, 1.8] 3.0 3.0
[6.8, 3.2, 5.9, 2.3] 3.0 3.0
[6.9, 3.1, 5.1, 2.3] 3.0 3.0
